In [2]:
import json

# 入力ファイル（あなたの元データ）のパス
INPUT_JSON = "test0_finetune_data_oneqa.json"
# 出力ファイル（LLaVA OneVision 用 JSONL）のパス
OUTPUT_JSONL = "test0_finetune_data_onevision.jsonl"

def convert_entry(entry):
    """
    元の entry を LLaVA OneVision 用フォーマットに変換する。
    """
    return {
        # サンプル識別子として id を source に
        "source": entry["id"],
        # 画像はリストに入れる
        "images": [entry["image"]],
        # 動画は今回は使わないので空リスト
        "videos": [],
        # system_prompt は未使用なので None
        "system_prompt": None,
        # 会話テキストをユーザ→アシスタントの順で並べたリストに
        "conversations": [
            entry["conversations"][0]["value"],
            entry["conversations"][1]["value"],
        ]
    }

def main():
    # 元データを読み込む
    with open(INPUT_JSON, "r", encoding="utf-8") as f:
        data = json.load(f)

    # JSONL に書き出す
    with open(OUTPUT_JSONL, "w", encoding="utf-8") as fout:
        for entry in data:
            new_entry = convert_entry(entry)
            fout.write(json.dumps(new_entry, ensure_ascii=False) + "\n")

    print(f"Converted {len(data)} entries to {OUTPUT_JSONL}")

if __name__ == "__main__":
    main()


Converted 33 entries to test0_finetune_data_onevision.jsonl
